In [1]:
import numpy as np
from osgeo import ogr, gdal, osr
import rasterio
import json


import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

#### Resize DEM from 30 to 10m

In [2]:
# Reading Normal DEM data
area_dem = '/home/shrayank_mistry/Modules/Area-routes/PanvelD/dem_clipped.tif'
dem = rasterio.open(area_dem, count = 1)

dem = np.array(dem.read(1))
print(dem.shape)

(6850, 3787)


In [3]:
# Up-sampling DEM to 10m resolution
import rasterio
from rasterio.enums import Resampling

# whole-numbers indicate upscaling, fractions indicate downscaling
upscale_factor = 3


with rasterio.open('/home/shrayank_mistry/Modules/Area-routes/PanvelD/dem_clipped.tif') as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [4]:
dem = data[0]
print(dem.shape)

(20550, 11361)


In [5]:
# reading LULC data
area_cover = '/home/shrayank_mistry/Modules/Area-routes/PanvelD/mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

In [7]:
dem_data, mask_data = dem, area

print(dem_data.shape, mask_data.shape)
width, height = 10980, 20550

dem_data, mask_data = dem_data[:height,:width], mask_data[:height, :width]
print(dem_data.shape, mask_data.shape)

dem_data, mask_data = dem_data.T, mask_data.T
print(dem_data.shape, mask_data.shape)

(20550, 11361) (20976, 10980)
(20550, 10980) (20550, 10980)
(10980, 20550) (10980, 20550)


#### AHP Mapping

In [8]:
ahp_map = {

'urban': 0.29,
'farms': 0.239,
'dense-forest': 0.207,
'water': 0.13,
'fallow': 0.067,
'sparse-forest': 0.049,
'barren-land': 0.019,
'unclassified':7,

}

class_map = {

    0: 'unclassified',
    1: 'water',
    2: 'dense-forest',
    3: 'sparse-forest',
    4: 'barren-land',
    5: 'urban',
    6: 'farms',
    7: 'fallow',

}

In [9]:
# Helper function to map from LULC to class-map to ahp values
def set_weights(c):
    c_str = class_map.get(c)
    return ahp_map[c_str]

In [10]:
set_weights_vtr = np.vectorize(set_weights)
mask_data = set_weights_vtr(mask_data)

In [11]:
np.unique(mask_data, return_counts = True)

(array([0, 7]), array([225632095,      6905]))

#### Start-End Co-ordinates

In [13]:
# ----------------------------- source_shp -------------------------------- #

file = ogr.Open("/home/shrayank_mistry/Modules/Area-routes/PanvelD/source/source.shp")
source_shp = file.GetLayer(0)

feature = source_shp.GetFeature(0)
source_shp = feature.ExportToJson()

source_shp = json.loads(source_shp)
start_ext = source_shp['geometry']['coordinates']

# ----------------------------- destination_shp -------------------------------- #

file = ogr.Open("/home/shrayank_mistry/Modules/Area-routes/PanvelD/destination/destination.shp")
destination_shp = file.GetLayer(0)

feature = destination_shp.GetFeature(0)
destination_shp = feature.ExportToJson()

destination_shp = json.loads(destination_shp)
end_ext = destination_shp['geometry']['coordinates']

In [14]:
print(start_ext, end_ext) 

[306666.8831168013, 2098263.628266526] [304717.23743578803, 1941342.757008533]


#### Raster Co-ordinates (Left, Bottom, Right, Top)

In [15]:
path = '/home/shrayank_mistry/Modules/Area-routes/PanvelD/raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=300000.0, bottom=1890240.0, right=409800.0, top=2100000.0)
300000.0 1890240.0 409800.0 2100000.0


In [16]:
width = round(right - left)
height = round(top - bottom)

print("Height and Width of Raster")
print(height, width)

Height and Width of Raster
209760 109800


### Finding pixels of start-point and end-point from extents

In [17]:
# [Do not run - already avaliable]
# start_pixel, end_pixel = [14424, 17699], [54850, 83443]
start_pixel, end_pixel = [0, 0], [0, 0]
start_flag, end_flag = True, True

for i in range(height):
    for j in range(width):

        if (not start_flag) and (not end_flag):
            break
        
        # print(round(left + j), round(top - i))
        if (start_flag and ((round(left + j)) == round(start_ext[0])) and ((round(top - i)) == round(start_ext[1]))):
            start_pixel = [i, j]
            start_flag = False

        
        if (end_flag and ((round(left + j)) == round(end_ext[0])) and ((round(top - i)) == round(end_ext[1]))):
            end_pixel = [i, j]
            end_flag = False

In [18]:
start_pixel, end_pixel 

([1736, 6667], [158657, 4717])

In [146]:
# start_pixel, end_pixel = [14424, 17699], [54850, 83443]

In [147]:
start_pixel = np.array(start_pixel)
start_pixel = start_pixel / 10

start_pixel = list(np.rint(start_pixel))
start_pixel = list(np.array(start_pixel, dtype = 'int'))

end_pixel = np.array(end_pixel)
end_pixel = end_pixel / 10

end_pixel = list(np.rint(end_pixel))
end_pixel = list(np.array(end_pixel, dtype = 'int'))

In [148]:
print(start_pixel, end_pixel)
start_pixel, end_pixel = [1770, 1442], [8344, 5485]
print(start_pixel, end_pixel)

[1442, 1770] [5485, 8344]
[1770, 1442] [8344, 5485]


#### A* search algorithm

In [149]:
def condition_check(start, end):
    if (start[0] == end[0]) and (start[1] == end[1]):
        return False
    return True

In [150]:
import math
from queue import PriorityQueue

In [151]:
mask_data = mask_data * 5000

In [152]:
np.unique(mask_data, return_counts = True)

(array([   95.,   245.,   335.,   650.,  1035.,  1195.,  1450., 35000.]),
 array([19822048, 17097029, 28893954,  7807154,  3858436,  5940554,
         2291737,      588]))

In [153]:
def c_anist_cost(i, j, x, y, mu = 10, wt = 2):

    mu_sqr = mu * mu
    h_diff = dem_data[i][j] - dem_data[x][y]
    h_sqr = h_diff * h_diff
    c_dv = (mask_data[i][j] + mask_data[x][y]) / 2
    cst = np.sqrt(mu_sqr + h_sqr) * (c_dv + math.atan(h_diff / mu) * wt) + acc_cost[i][j]

    return cst

In [154]:
# Manhattan Distance
def h_cost_manhattan(i, j):
    xa, ya, xb, yb = end_pixel[0], end_pixel[1], i, j
    return abs(xa - xb) + abs(ya - yb)

# Euclidean Distance
def h_cost_euclidean(i, j):
    xa, ya, xb, yb = end_pixel[0], end_pixel[1], i, j
    return math.sqrt((xa - xb) ** 2 + (ya - yb) ** 2)

# Diagonal Distance
def h_cost_diagonal(i, j):
    xa, ya, xb, yb = end_pixel[0], end_pixel[1], i, j
    return abs(xa - xb) + abs(ya - yb) + (math.sqrt(2) - 2) * min(abs(xa - xb), abs(ya - yb))

# Chebyshev Distance
def h_cost_chebyshev(i, j):
    xa, ya, xb, yb = end_pixel[0], end_pixel[1], i, j
    return max(abs(xa - xb), abs(ya - yb))

# New-heuristic function
def h_cost_new(i, j):
    xa, ya, xb, yb = end_pixel[0], end_pixel[1], i, j
    return h_cost_euclidean(i, j) + h_cost_chebyshev(i, j) * 2

In [155]:
heuristic_dict = {
    1: h_cost_manhattan,
    2: h_cost_euclidean,
    3: h_cost_diagonal,
    4: h_cost_chebyshev,
    5: h_cost_new
}

In [156]:
def get_neigh_cost(i, j, flag):
    arr = []

    heuristic_func = heuristic_dict.get(flag)


    #(1) col - 1, row - 1
    if (j - 1 >= 0) and (i - 1 >= 0):
        g = c_anist_cost(i, j, i - 1, j - 1)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i - 1][j - 1] > (g + h)):
            parent[i - 1][j - 1] = i, j
            acc_cost[i - 1][j - 1] = (g + h)
        arr.append([acc_cost[i - 1][j - 1], [i - 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(2) col, row - 1
    if (i - 1 >= 0):
        g = c_anist_cost(i, j, i - 1, j)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i - 1][j] > (g + h)):
            parent[i - 1][j] = i, j
            acc_cost[i - 1][j] = (g + h)
        arr.append([acc_cost[i - 1][j], [i - 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(3) col + 1, row - 1
    if (j + 1 < 8163) and (i - 1 >= 0):
        g = c_anist_cost(i, j, i - 1, j + 1)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i - 1][j + 1] > (g + h)):
            parent[i - 1][j + 1] = i, j
            acc_cost[i - 1][j + 1] = (g + h)
        arr.append([acc_cost[i - 1][j + 1], [i - 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(4) col - 1, row
    if (j - 1 >= 0):
        g = c_anist_cost(i, j, i, j - 1)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i][j - 1] > (g + h)):
            parent[i][j - 1] = i, j
            acc_cost[i][j - 1] = (g + h)
        arr.append([acc_cost[i][j - 1], [i, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(5) col + 1, row
    if (j + 1 < 8163):
        g = c_anist_cost(i, j, i, j + 1)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i][j + 1] > (g + h)):
            parent[i][j + 1] = i, j
            acc_cost[i][j + 1] = (g + h)
        arr.append([acc_cost[i][j + 1], [i, j + 1], [i, j]])
    else:
        arr.append(math.inf)

    #(6) col - 1, row + 1
    if (j - 1 >= 0) and (i + 1 < 10500):
        g = c_anist_cost(i, j, i + 1, j - 1)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i + 1][j - 1] > (g + h)):
            parent[i + 1][j - 1] = i, j
            acc_cost[i + 1][j - 1] = (g + h)
        arr.append([acc_cost[i + 1][j - 1], [i + 1, j - 1], [i, j]])
    else:
        arr.append(math.inf)

    #(7) col, row + 1
    if (i + 1 < 10500):
        g = c_anist_cost(i, j, i + 1, j)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i + 1][j] > (g + h)):
            parent[i + 1][j] = i, j
            acc_cost[i + 1][j] = (g + h)
        arr.append([acc_cost[i + 1][j], [i + 1, j], [i, j]])
    else:
        arr.append(math.inf)

    #(8) col + 1, row + 1
    if (j + 1 < 8163) and (i + 1 < 10500):
        g = c_anist_cost(i, j, i + 1, j + 1)
        h = heuristic_func(i - 1, j - 1)

        if (acc_cost[i + 1][j + 1] > (g + h)):
            parent[i + 1][j + 1] = i, j
            acc_cost[i + 1][j + 1] = (g + h)
        arr.append([acc_cost[i + 1][j + 1], [i + 1, j + 1], [i, j]])
    else:
        arr.append(math.inf)
    

    return arr

In [157]:
end_pixel

[8344, 5485]

In [158]:
dim = mask_data.shape
print(dim)

(10500, 8163)


In [159]:
start_pixel, end_pixel = [1770, 1442], [8344, 5485]
# start_pixel, end_pixel = [1770, 1442], [1980, 1600]
print(start_pixel, end_pixel)

[1770, 1442] [8344, 5485]


### Heuristic Check Flag Setting
#### 1. Manhattan
#### 2. Euclidean
#### 3. Diagonal distance heuristic function
#### 4. Chebyshev distance heuristic function
#### 5. new heuristic function

In [332]:
FLAG = 5

In [333]:
Q = PriorityQueue()
s_pixel, e_pixel = start_pixel, end_pixel

# mask_copy.shape, dem_data.shape

visited = np.zeros((dim))
acc_cost = np.full((dim), math.inf)

acc_cost[s_pixel[0]][s_pixel[1]] = 0
visited[s_pixel[0]][s_pixel[1]] = 1

parent = np.full((dim), None)
parent[s_pixel[0]][s_pixel[1]] = -1, -1

while (condition_check(s_pixel, e_pixel)):
    
    i, j = s_pixel
    # h-cost(0) - indicates Euclidean distance
    # h-cost(1) - indicates Manhattan distance

    neighbours_cost = get_neigh_cost(i, j, FLAG)
    for nc in neighbours_cost:
        if nc == math.inf:
            continue
        else:
            Q.put(nc)
    
    bst = Q.get()
    m, n = bst[1][0], bst[1][1]

    while True:
        if visited[m][n] == 0:
            break
        bst = Q.get()
        m, n = bst[1][0], bst[1][1]

    # set-visited
    visited[m][n] = 1
    # path.append([m, n])
    # print(m, n)

    parent[m][n] = bst[2][0], bst[2][1]

    s_pixel = [m, n]

In [334]:
print(acc_cost[end_pixel[0]][end_pixel[1]])

66997309.37078367


In [335]:
!rm -rf path.txt

In [ ]:
!mkdir f'paths/P{FLAG}'

In [336]:
path = []
pr = parent[end_pixel[0]][end_pixel[1]]

# cnt = 15000
while (pr[0] != -1) and (pr[1] != -1):
    path.append(pr)
    # path.append('-')
    with open(f'paths/P{FLAG}/path.txt', 'a') as f:
        f.write(str(pr) + '\n')

    # cnt = cnt - 1
    # if cnt == 0:
    #     break

    pr = parent[pr[0]][pr[1]]

In [337]:
path_list = []
with open(f'paths/P{FLAG}/path.txt', 'r') as f:
    for point in f:
        path_list.append(point)

In [338]:
len(path_list)

6787

In [339]:
point = path_list[0].replace('\n', '').split(' ')
pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])
print(pi, pj)

8343 5484


In [340]:
len(path_list)

6787

In [341]:
path_list.reverse()

In [342]:
%%time
ordinates_dict = {}


for i in range(len(path_list)):
    point = path_list[i].replace('\n', '').split(' ')
    pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

    ext_i, ext_j = left + (pi * 10), top - (pj * 10)

    # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
    ordinates_dict[pi, pj] = [ext_i, ext_j]

CPU times: user 42.8 ms, sys: 28 ms, total: 70.8 ms
Wall time: 70.4 ms


In [343]:
ordinates_dict
ordinates_list = []
for key, value in ordinates_dict.items():
    ordinates_list.append(value)

### Heuristic Check Flag Setting
#### 1. Manhattan
#### 2. Euclidean
#### 3. Diagonal distance heuristic function
#### 4. Chebyshev distance heuristic function
#### 5. new heuristic function

In [344]:
route_paths = {
    1: 'a-star-man',
    2: 'a-star-euc',
    3: 'a-star-diag',
    4: 'a-star-chesb',
    5: 'a-star-new',
}

In [345]:
import shapefile
w = shapefile.Writer(f'/home/shrayank_mistry/Modules/Area-routes/PanelD/routes-shape/{route_paths[FLAG]}/shapefiles/test/multipoint')
w.field('name', 'C')

w.multipoint(ordinates_list) 
w.record('multipoint1')

w.close()

In [346]:
road_lenght = 0
index = 1
for _ in ordinates_list[1:]:
    i, j = ordinates_list[index][0], ordinates_list[index][1]
    x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

    if (x - i == 10.0) and (y - j == -10.0):
        road_lenght += math.sqrt(2 * 100)
    else:
        road_lenght += 10 

In [347]:
print(f'Current Road-length = {road_lenght / 1000} kms')

Current Road-length = 95.96853234263206 kms


#### Co-ordinates to Pixel Mapping

In [226]:
# %%time
# pixels_dict = {}


# for i in range(len(path_list)):
#     point = path_list[i].replace('\n', '').split(' ')
#     pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

#     ext_i, ext_j = left + (pi * 10), top - (pj * 10)

#     # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
#     # ordinates_dict[pi, pj] = [ext_i, ext_j]
#     pixels_dict[ext_i, ext_j] = [pi, pj]

CPU times: user 216 ms, sys: 8 ms, total: 224 ms
Wall time: 228 ms


In [231]:
# # reading LULC data
# area_cover = '/home/shrayank_mistry/Modules/project-mum-pune/mask.tif'
# area = rasterio.open(area_cover, count = 1)

# area = np.array(area.read(1))

# dem_data, mask_data = dem, area

# print(dem_data.shape, mask_data.shape)
# width, height = 10500, 8163

# dem_data, mask_data = dem_data[:height,:width], mask_data[:height, :width]
# print(dem_data.shape, mask_data.shape)

# dem_data, mask_data = dem_data.T, mask_data.T
# print(dem_data.shape, mask_data.shape)

(8163, 10830) (8273, 10500)
(8163, 10500) (8163, 10500)
(10500, 8163) (10500, 8163)


In [312]:
# check_pixels = {
#     1: 0,
#     2: 0,
#     3: 0,
#     4: 0,
#     5: 0,
#     6: 0,
#     7: 0,
# }


In [313]:
# r = shapefile.Reader(f'/home/shrayank_mistry/Modules/project-mum-pune/routes-shape/{route_paths[FLAG]}/shapefiles/test/multipoint.shp')
# feature = r.shapeRecords()[0]
# first = feature.shape.__geo_interface__  

In [314]:
# total_pxs = len(first['coordinates'])
# for cord in first['coordinates']:
#     pi, pj = pixels_dict[cord]
#     cs = mask_data[pi][pj]

#     check_pixels[cs] += 1

In [315]:
# check_pixels

{1: 593, 2: 502, 3: 1541, 4: 2075, 5: 263, 6: 179, 7: 2445}

#### Percentage Calculation

In [316]:
# for key, value in check_pixels.items():
#     check_pixels[key] = np.round(((value / total_pxs) * 100), 2)

In [317]:
# check_pixels

{1: 7.8, 2: 6.61, 3: 20.28, 4: 27.31, 5: 3.46, 6: 2.36, 7: 32.18}

In [320]:
# classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland'}
# df_dict_perc = {}
# for key, value in check_pixels.items():
#     df_dict_perc[classes[key]] = f'{check_pixels[key]}%'


In [321]:
# df_dict_perc

{'Water': '7.8%',
 'Dense-forest': '6.61%',
 'Sparse-forest': '20.28%',
 'Barren-land': '27.31%',
 'Urban-land': '3.46%',
 'Farmland': '2.36%',
 'Fallowland': '32.18%'}

In [297]:
# print(f'Current Road-length = {road_lenght / 1000} kms')

Current Road-length = 75.97 kms


In [300]:
# road_distance = road_lenght / 1000
# road_distance

75.97

In [327]:
# df_dict_dist = {}
# recheck = 0
# for key, value in check_pixels.items():
#     df_dict_dist[classes[key]] = (check_pixels[key] * road_distance) / 100
#     recheck += (check_pixels[key] * road_distance) / 100

In [330]:
# df_dict_dist

{'Water': 5.925660000000001,
 'Dense-forest': 5.021617,
 'Sparse-forest': 15.406716000000001,
 'Barren-land': 20.747407,
 'Urban-land': 2.628562,
 'Farmland': 1.792892,
 'Fallowland': 24.447145999999996}

In [331]:
# recheck

75.97